# Long-context LLMs
We use long-context LLMs compared with RAG-based methods , utilizing `GPT-4-Turbo` with a 128k context window and `Qwen-1.5-7B` with a 32k context window.

In [1]:
# Preparation
import sys
import os
from pathlib import Path

# Get the project root directory
root_dir = Path(os.path.abspath("")).resolve().parents[1]
sys.path.append(str(root_dir))
# Change the working directory to the project root
os.chdir(root_dir)

# Set up the configs for this demo
res_dir = f"experiment/long_context/res/"
if not os.path.exists(res_dir):
    os.makedirs(res_dir)

In our paper, we use the powerful `text-embedding-3-large-model` model with AzureOpenAI-API . But you need to set up with your own api-key, endpoint and deploy-model in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). 

If you want to use the API from **other alternative platforms** please change the codes in [uda/utils/retrieve.py (line-80)](../../uda/utils/retrieve.py#L81). 

For convenient demonstration, we choose the `BM25` retriever here.

In [2]:
# Experimental Configurations

# Available retrieval model_name: "bm25", "all-MiniLM-L6-v2", "all-mpnet-base-v2", "openai", "colbert"
# We choose bm25 for convenience
RT_MODEL = "bm25" 


DATASET_NAME_LIST = ["paper_text", "fin", "paper_tab",  "feta", "nq", "fin"]

LOCAL_LLM_DICT = {"qwen-7B": "Qwen/Qwen1.5-7B-Chat"}
LLM_LIST = ["gpt4", "qwen-7B"]

# Sample a subset of LLMs for faster demo
DATASET_NAME_LIST = DATASET_NAME_LIST[:2]
LLM_LIST = LLM_LIST[:1]
DEMO_DOC_NUM = 2
DEMO_QA_NUM = 2

We utilized the Qwen-1.5-7B-32k and GPT4-Turbo-128k models, capable of processing larger text blocks. However, it may still be insufficient when dealing with quite lengthy documents, like financial reports that surpass 100k words (over 150k tokens). 

For such cases, full-context will reverts to a retrieval-based approach, selecting the 30 most relevant chunks to construct the input context.

In [3]:
# Functions to implement full-context policy
from uda.utils.retrieve import extract_text_from_pdf
from uda.utils import preprocess as pre
from transformers import AutoTokenizer

def __get_token_length(text):
    # use the llama tokenizer to calculate the approximate token length
    model_name = "meta-llama/Meta-Llama-3-8B-Instruct" 
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        trust_remote_code=True,
        resume_download=True,
        use_fast=True,
    )
    token_res = tokenizer(text)["input_ids"]
    return len(token_res)


def get_full_context(pdf_path, llm_type):
    if llm_type == "gpt4":
        CONTEXT_SIZE = 127 * 1024  # 1k tokens for prompt
    if llm_type == "qwen-7B":
        CONTEXT_SIZE = 31 * 1024
    text = extract_text_from_pdf(pdf_path)
    token_length = __get_token_length(text)
    if token_length < CONTEXT_SIZE:
        return text
    else:       
        print(f"Token length of {pdf_path}: {token_length}, exceeds the limit. Full-context falls back to top-30 relevant chunks.") 
        return None # the caller code will implement top-30 RAG policy

We use the **AzureOpenAI-API** serves as the interface for accessing GPT models. Users should set up the gpt-service with their own api-key and endpoint in the config_file [uda/utils/access_config.py](../../uda/utils/access_config.py). These configurations will be used in the `call_gpt()` function in the following codes.


If you want to use **other alternative platforms**, the `call_gpt()` can be replaced by the corresponding model-calling function.

### Local Inference with Long-Context LLMs

Additionally, executing local inference on long-context LLMs can be memory-intensive, often necessitating significant GPU memory.

 To alleviate this, you should install the `flashattention` [library](https://github.com/Dao-AILab/flash-attention). This optimization can be enabled by adding the parameter `attn_implementation="flash_attention_2"` within the `inference.py` script, which can be found at [uda/utils/inference.py, line 27](../../uda/utils/inference.py#L27).

In [4]:
from uda.utils import retrieve as rt
from uda.utils import preprocess as pre
import pandas as pd
from uda.utils import llm
from uda.utils import inference
import json

for DATASET_NAME in DATASET_NAME_LIST:
    for LLM_MODEL in LLM_LIST:
        print(f"=== Start {DATASET_NAME} on {LLM_MODEL} ===")
        res_file = os.path.join(res_dir, f"{DATASET_NAME}_{LLM_MODEL}_{RT_MODEL}.jsonl")

        # If use the local LLM, initialize the model
        if LLM_MODEL in LOCAL_LLM_DICT:
            llm_name = LOCAL_LLM_DICT[LLM_MODEL]
            llm_service = inference.LLM(llm_name)
            llm_service.init_llm()

        # Load the benchmark data
        bench_json_file = pre.meta_data[DATASET_NAME]["bench_json_file"]
        with open(bench_json_file, "r") as f:
            bench_data = json.load(f)

        # Run experiments on the demo docs
        doc_list = list(bench_data.keys())
        for doc in doc_list[:DEMO_DOC_NUM]:
            pdf_path = pre.get_example_pdf_path(DATASET_NAME, doc)
            if pdf_path is None:
                continue

            # Get full context
            full_context = get_full_context(pdf_path, LLM_MODEL)
            # If exceed the context window, fall back to the RAG with Top30 chunks
            if full_context is None:
                # Prepare the index for RAG
                collection_name = f"{DATASET_NAME}_vector_db"
                collection = rt.prepare_collection(pdf_path, collection_name, RT_MODEL)

            for qa_item in bench_data[doc][:DEMO_QA_NUM]:
                question = qa_item["question"]
                q_uid = qa_item["q_uid"]
                if full_context is None:
                    # Retrieve the Top30 contexts
                    contexts = rt.get_contexts(collection, question, RT_MODEL,top_k=30)
                    context_text = '\n'.join(contexts)
                else:
                    context_text = full_context
                # Create the prompt
                llm_message = llm.make_prompt(question, context_text, DATASET_NAME, LLM_MODEL)
                # Generate the answer
                if LLM_MODEL in LOCAL_LLM_DICT:
                    response = llm_service.infer(llm_message)
                elif LLM_MODEL == "gpt4":
                    # Set up with your own GPT4 service
                    response = llm.call_gpt(messages=llm_message)
                    if response is None:
                        print("Make sure your gpt4 service is set up correctly.")
                        raise Exception("GPT4 service")

                # log the results
                res_dict = {"model": LLM_MODEL, "question": question, "response": response, "doc": doc, "q_uid": q_uid, "answers": qa_item["answers"]}
                print(res_dict)
                with open(res_file, "a") as f:
                    f.write(json.dumps(res_dict) + "\n")
            
            if full_context is None:
                rt.reset_collection(collection_name, RT_MODEL)
            

    print(f"=== Finish {DATASET_NAME} ===\n")


=== Start paper_text on gpt4 ===


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-07-05 12:39:43,856 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'Do they perform error analysis?', 'response': 'The answer is: No, they do not perform error analysis.', 'doc': '2001.03131', 'q_uid': '133eb4aa4394758be5f41744c60c99901b2bc01c', 'answers': [{'answer': 'No', 'type': 'boolean'}, {'answer': 'No', 'type': 'boolean'}]}


2024-07-05 12:39:51,074 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'What is the Random Kitchen Sink approach?', 'response': 'The answer is: The Random Kitchen Sink (RKS) approach is a method that explicitly maps data vectors to a space where linear separation is possible, and in that space, regularized least-square based classification (RLSC) is performed.', 'doc': '2001.03131', 'q_uid': 'a778b8204a415b295f73b93623d09599f242f202', 'answers': [{'answer': 'Random Kitchen Sink method uses a kernel function to map data vectors to a space where linear separation is possible.', 'type': 'abstractive'}, {'answer': 'explicitly maps data vectors to a space where linear separation is possible, RKS method provides an approximate kernel function via explicit mapping', 'type': 'extractive'}]}


...

{'model': 'gpt4', 'question': 'what percentage of total long-term assets under supervision are comprised of fixed income in 2016?', 'response': 'The total long-term assets under supervision in 2016 are $1,021 billion, and the fixed income assets under supervision are $601 billion. To calculate the percentage:\n\n(601 / 1,021) * 100 = 58.86%\n\nThe answer is: 58.86%', 'doc': 'GS_2016', 'q_uid': 'GS/2016/page_79.pdf-1', 'answers': {'str_answer': '59%', 'exe_answer': 0.588}}


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-07-05 12:40:41,081 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'what was the percentage change in the 5 year annual performance of the peer group stock from 2010 to 2011', 'response': 'The percentage change in the 5-year annual performance of the peer group stock from 2010 to 2011 was 36.78% - 100.00% = -63.22%.\n\nThe answer is: -63.22%', 'doc': 'JKHY_2015', 'q_uid': 'JKHY/2015/page_20.pdf-2', 'answers': {'str_answer': '8.3%', 'exe_answer': 0.08276}}


2024-07-05 12:40:52,010 - INFO - HTTP Request: POST https://qinchuan-hui.openai.azure.com/openai/deployments/yulong-4/chat/completions?api-version=2024-04-01-preview "HTTP/1.1 200 OK"


{'model': 'gpt4', 'question': 'in 2010 , what was the cumulative total return of the s&p 500?', 'response': 'The cumulative total return of the S&P 500 in 2010 was 130.69. \n\nThe answer is: 130.69', 'doc': 'JKHY_2015', 'q_uid': 'JKHY/2015/page_20.pdf-3', 'answers': {'str_answer': '30.69', 'exe_answer': 30.69}}
=== Finish fin ===



### Evaluate the generating results

In [5]:
dataset_name="paper_text"
llm_model="gpt4"
rt_model="bm25"
res_file_name=f"experiment/long_context/res/{dataset_name}_{llm_model}_{rt_model}.jsonl"

from uda.eval.my_eval import eval_from_file
eval_from_file(dataset_name, res_file_name)

{'Answer F1': 0.6114498644986449, 'Missing predictions': 0}
